In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

!pip install pandas-datareader
import pandas_datareader as pdr

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import gridspec
!pip install mplfinance
from mplfinance.original_flavor import candlestick2_ohlc

import datetime
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
class Stocks:
    def __init__(self, symbol, start_day, end_day):
        self.symbol = symbol
        self.start_day = start_day
        self.end_day = end_day

    def get_price_data(self):
        #df_price = pdr.get_data_yahoo(symbol, self.start_day - timedelta(days=365), self.end_day)
        df_price = pdr.DataReader(self.symbol, 'yahoo',self.start_day, self.end_day)

        return df_price

    def with_moving_ave(self):
        df_price = pdr.DataReader(self.symbol, 'yahoo',self.start_day, self.end_day)
        ma5= df_price['Adj Close'].rolling(window = 5, min_periods=1).mean()
        ma20= df_price['Adj Close'].rolling(window = 20, min_periods=1).mean()
        ma60= df_price['Adj Close'].rolling(window = 60, min_periods=1).mean()
        ma120= df_price['Adj Close'].rolling(window = 120, min_periods=1).mean()
        std = df_price['Adj Close'].rolling(window = 20, min_periods=1).std()
        bol_upper = ma20 + 2*std
        bol_down = ma20 - 2*std

        df_price.insert(len(df_price.columns), 'MA5', ma5)
        df_price.insert(len(df_price.columns), 'MA20', ma20)
        df_price.insert(len(df_price.columns), 'MA60', ma60)
        df_price.insert(len(df_price.columns), 'MA120', ma120)
        df_price.insert(len(df_price.columns), 'Std', std)
        df_price.insert(len(df_price.columns), 'bol_upper', bol_upper)
        df_price.insert(len(df_price.columns), 'bol_down', bol_down)

        return df_price

In [ ]:
td_1y = datetime.timedelta(weeks=52/2)
#start_day = datetime.datetime(2019,1,1)
#end_day = datetime.datetime(2021,5,1)
today = datetime.datetime.now()
start_day = today - td_1y

# To test just a ticker for making class
# symbol = input('Write ticker name like aapl: ')
symbol = 'AAPL' # example :)

stock = Stocks(symbol, start_day, today)

In [ ]:
# For figure
df = stock.with_moving_ave()
index = df.index.astype('str')
fig = plt.figure(figsize=(10,10))

gs = gridspec.GridSpec(2,1,height_ratios=[3,1])
ax_main = plt.subplot(gs[0])
ax_1 = plt.subplot(gs[1])

def x_date(x,pos):
    try:
        return index[int(x-0.5)][:7]
    except IndexError:
        return ''

# ax_main
ax_main.xaxis.set_major_locator(ticker.MaxNLocator(10))
ax_main.xaxis.set_major_formatter(ticker.FuncFormatter(x_date))

ax_main.set_title(symbol+' stock', fontsize=22 )
#ax_main.set_xlabel('Date')
ax_main.plot(index, df['MA5'], label='MA5')
ax_main.plot(index, df['MA20'], label='MA20')
ax_main.plot(index, df['MA60'], label='MA60')
ax_main.plot(index, df['MA120'], label='MA120')
ax_main.plot(index, df['bol_upper'], label='bol_upper')
ax_main.plot(index, df['bol_down'], label='bol_down')
ax_main.fill_between(index, df['bol_down'], df['bol_upper'], color = 'gray')

candlestick2_ohlc(ax_main,df['Open'],df['High'],df['Low'],df['Close'], width=0.5, colorup='r', colordown='b')

ax_main.legend(loc=2)

# ax_1
ax_1.xaxis.set_major_locator(ticker.MaxNLocator(10))
ax_1.xaxis.set_major_formatter(ticker.FuncFormatter(x_date))
ax_1.set_xlabel('Date')
ax_1.set_ylabel('Std')
ax_1.plot(index, df['Std'], label='Std')
mean = df['Std'].mean()
ax_1.hlines(y=mean, xmin=index[0], xmax=index[-1], colors='red')

plt.grid()
plt.show()

If you are satisfied in data and code, please upvote :)👍 

Welcom a lot of comment!